In [4]:
import pandas as pd
import os
import multiprocessing
from concurrent.futures import ProcessPoolExecutor, as_completed

# CORREÇÃO 1: Importar a versão "Standard" (texto) para não dar erro de Widget
from tqdm.std import tqdm 

# Importa sua função
from extractor import extrair_todos_grupos_final

if __name__ == '__main__':
    
    # --- DIAGNÓSTICO DE CAMINHO (Para sabermos onde estamos) ---
    diretorio_atual = os.getcwd()
    print(f"1. O Notebook está rodando na pasta: {diretorio_atual}")
    
    # --- CONFIGURAÇÃO DO CAMINHO ---
    # Tenta subir um nível (..) e entrar em data/input_pdfs
    # Se o notebook está em 'src', isso deve levar para 'etl-pdf.../data/input_pdfs'
    pasta_origem = os.path.abspath(os.path.join(diretorio_atual, "..", "data", "input_pdfs"))
    
    print(f"2. O Script vai procurar os PDFs em: {pasta_origem}")
    
    # Validação
    if not os.path.exists(pasta_origem):
        print("\n❌ ERRO: A pasta ainda não foi encontrada.")
        print("TENTE ISTO: Copie o caminho COMPLETO da pasta 'input_pdfs' do seu Windows explorer")
        print("e cole na variável 'pasta_origem' abaixo, substituindo o código automático.")
        
        # EXEMPLO DE CORREÇÃO MANUAL (Descomente e ajuste se precisar):
        # pasta_origem = r"C:\Users\SeuUsuario\Scripts Python\etl-pdf-multiprocessamento\data\input_pdfs"
    
    else:
        print("✅ SUCESSO: Pasta encontrada! Iniciando trabalhos...\n")
        
        # --- NOMES DE SAÍDA ---
        pasta_saida = os.path.join(os.path.dirname(pasta_origem), 'output')
        if not os.path.exists(pasta_saida): os.makedirs(pasta_saida)
        
        arquivo_saida = os.path.join(pasta_saida, "Relatorio_Final.csv")
        arquivo_log = os.path.join(pasta_saida, "Log_Erros.txt")

        # --- LISTAGEM ---
        arquivos_pdf = [f for f in os.listdir(pasta_origem) if f.lower().endswith('.pdf')]
        caminhos_completos = [os.path.join(pasta_origem, f) for f in arquivos_pdf]
        total = len(caminhos_completos)
        
        num_processos = max(1, multiprocessing.cpu_count() - 1)
        print(f"Arquivos PDF encontrados: {total}")
        
        if total > 0:
            # --- PREPARA CSV ---
            colunas = [
                "Arquivo", "Grupo", 
                "Fase_Pickup", "Fase_Seq", "Fase_Lenta_Curva", "Fase_Lenta_Dial", "Fase_Lenta_TAdic",
                "Fase_Rapida_Curva", "Fase_Rapida_Dial", "Fase_Rapida_TAdic",
                "Terra_Pickup", "Terra_Seq", "Terra_Lenta_Curva", "Terra_Lenta_Tempo",
                "Terra_Rapida_Curva", "Terra_Rapida_Tempo", "Tempo_Morto"
            ]
            pd.DataFrame(columns=colunas).to_csv(arquivo_saida, index=False, sep=';', encoding='utf-8-sig')

            # --- EXECUÇÃO PARALELA ---
            buffer = []
            lote_gravacao = 100 

            print(f"Iniciando Motor Turbo com {num_processos} núcleos...")
            
            with ProcessPoolExecutor(max_workers=num_processos) as executor:
                futures = {executor.submit(extrair_todos_grupos_final, c): c for c in caminhos_completos}
                
                # AQUI ESTÁ A CORREÇÃO DA BARRA: tqdm padrão
                for future in tqdm(as_completed(futures), total=total, desc="Processando", unit="pdf"):
                    try:
                        dados, erro = future.result()
                        if erro:
                            with open(arquivo_log, "a") as f: f.write(f"{erro}\n")
                        elif dados:
                            buffer.extend(dados)
                        
                        if len(buffer) >= lote_gravacao:
                            df_temp = pd.DataFrame(buffer)
                            for col in colunas:
                                if col not in df_temp.columns: df_temp[col] = ""
                            df_temp = df_temp[colunas]
                            df_temp.to_csv(arquivo_saida, mode='a', header=False, index=False, sep=';', encoding='utf-8-sig')
                            buffer = []
                    except Exception as e:
                        print(f"Erro genérico: {e}")

            if buffer:
                df_temp = pd.DataFrame(buffer)
                for col in colunas:
                    if col not in df_temp.columns: df_temp[col] = ""
                df_temp = df_temp[colunas]
                df_temp.to_csv(arquivo_saida, mode='a', header=False, index=False, sep=';', encoding='utf-8-sig')

            print("-" * 50)
            print("CONCLUÍDO! Verifique a pasta 'data/output'.")
        else:
            print("A pasta existe, mas não tem arquivos .pdf dentro dela.")

1. O Notebook está rodando na pasta: C:\Users\edu_m\Scripts Python\etl-pdf-multiprocessamento\src
2. O Script vai procurar os PDFs em: C:\Users\edu_m\Scripts Python\etl-pdf-multiprocessamento\data\input_pdfs
✅ SUCESSO: Pasta encontrada! Iniciando trabalhos...

Arquivos PDF encontrados: 50
Iniciando Motor Turbo com 15 núcleos...


Processando: 100%|████████████████████████████████████████████████████████████████████| 50/50 [00:01<00:00, 36.43pdf/s]


--------------------------------------------------
CONCLUÍDO! Verifique a pasta 'data/output'.


In [ ]:
!pip install ipywidgets